In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd

In [2]:
# x = np.arange(2,11).reshape(3,3)
x = np.random.normal(3,1,10).reshape(10,1)
e = np.random.normal(0,.5,10).reshape(10,1)
y = x + e
X = np.hstack((np.ones((10,1)),x))

In [3]:
df = np.shape(x)[1]+1 # degree of freedom = number of columns
n = len(y)  # number of rows
XTX = np.dot(X.T,X) # gramian matrix
XTY = np.dot(X.T,y) # moment matrix
Beta = np.dot(np.linalg.inv(XTX),XTY)  # beta values  *****
Yhat = Beta[0] + (Beta[1]*x)  # y hat
ehat = y - Yhat  # error
RSS = ehat.T.dot(ehat)
TSS = ((y - y.mean())**2).sum()
Rsq = 1 - (RSS/TSS)  # Rsquared  *****
Rsq_adj = 1-((1-Rsq)*((n-1)/(n-df)))
# var covar  - - - standard error of each beta coefficient is equal to square root of the j-th diagonal element of this matrix
# var(beta cap) = sigmasq* (XTX)^-1
sigmasq = (ehat**2).sum()/(n-df) # variance of parameters
var_beta = sigmasq * np.linalg.inv(XTX) # variance
se_beta = np.sqrt(np.diag(var_beta)) # standard error of parameters *****
tval = Beta/se_beta.reshape(2,1) # tvalue *****
pval = stats.t.pdf(tval,len(y)-df)  # pvalue ******does not match ???
tcrit = stats.t.ppf(1-0.025, n-df)
LCL = Beta - tcrit*se_beta.reshape(2,1)
UCL = Beta + tcrit*se_beta.reshape(2,1)

output = pd.DataFrame(np.hstack((Beta,se_beta.reshape(2,1),tval.reshape(2,1),pval.reshape(2,1),LCL.reshape(2,1),UCL.reshape(2,1))))
output.columns = ["Beta","Standard err","Tval","pval","LCL","UCL"]
print(output)
print("\nRsquared:"+ str(Rsq))
print("\nRsquared Adjusted:"+str(Rsq_adj))

       Beta  Standard err      Tval      pval       LCL       UCL
0 -0.012082      0.599629 -0.020149  0.386611 -1.394828  1.370664
1  0.988672      0.192457  5.137118  0.000546  0.544867  1.432478

Rsquared:[[0.76737408]]

Rsquared Adjusted:[[0.73829584]]


In [9]:
import statsmodels.api as sm
model = sm.OLS(y,sm.add_constant(x)).fit()
# model = sm.api.OLS(y,sm.api.add_constant(x)).fit()
model.summary()


C:\Users\165210\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:1394: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.767
Model:                            OLS   Adj. R-squared:                  0.738
Method:                 Least Squares   F-statistic:                     26.39
Date:                Wed, 08 Aug 2018   Prob (F-statistic):           0.000888
Time:                        23:10:07   Log-Likelihood:                -6.6406
No. Observations:                  10   AIC:                             17.28
Df Residuals:                       8   BIC:                             17.89
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0121      0.600     -0.020      0.984      -1.395       1.371
x1             0.9887      0.192      5.137      0.001       0.545       1.432
==============================================================================
Omnibus:                        0.131   Durbin-Watson:                   2.397
Prob(Omnibus):                  0.937   Jarque-Bera (JB):                0.299
Skew:                           0.192   Prob(JB):                        0.861
Kurtosis:                       2.245   Cond. No.                         12.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""